In [1]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text

import seaborn as sns
import random
import matplotlib.pyplot as plt


import re,string,unicodedata

import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from wordcloud import WordCloud,STOPWORDS
from bs4 import BeautifulSoup

from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daisuke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Data Cleaning

In [4]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)



# BeautifulSoup(解析対象のHTML/XML, 利用するパーサー)
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()



#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)



# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)




#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)



#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text


#Apply function on review column
df['review']=df['review'].apply(denoise_text)

In [5]:
# So clean

df.sentiment.replace("positive" , 1 , inplace = True)
df.sentiment.replace("negative" , 0 , inplace = True)
df.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ex...,1
1,wonderful little filming technique fashion giv...,1
2,thought wonderful way spend time hot summer si...,1
3,basically family little boy thinks zombie clos...,0
4,petter time visually stunning film mattei offe...,1


## 3 One hot encoding

In [6]:
# yの形状をone hot にする
type_one_hot = OneHotEncoder(sparse=False).fit_transform(df.sentiment.to_numpy().reshape(-1, 1))

In [8]:
type_one_hot

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [7]:
df.review

0        one reviewers mentioned watching oz episode ex...
1        wonderful little filming technique fashion giv...
2        thought wonderful way spend time hot summer si...
3        basically family little boy thinks zombie clos...
4        petter time visually stunning film mattei offe...
                               ...                        
49995    thought movie right good creative original exp...
49996    bad bad bad idiotic annoying porn groove sound...
49997    catholic taught parochial elementary schools t...
49998    going disagree previous comment side maltin se...
49999    one expects star trek movies high fans expect ...
Name: review, Length: 50000, dtype: object

In [28]:
train_reviews, test_reviews, y_train, y_test = train_test_split(df.review, 
                                                                type_one_hot, 
                                                                test_size=0.5, 
                                                                random_state=0)



In [29]:
print(train_reviews.shape, test_reviews.shape)

(25000,) (25000,)


In [30]:
train_reviews

21626    portrays day day stark reality survival ranch ...
4521     interesting study societal well say wife stron...
3652     prior airport movies rather decent special goo...
22212    without doubt exciting satisfying film seen pl...
17669    maybe alone crap sorry lured found bill idea c...
                               ...                        
21243    set high expectations left pleasantly story li...
45891    blob great horror merely vividly horrific imag...
42613    many years anne rivers noted book house next f...
43567    massive fan thought first two series provided ...
2732     ag excellent presentation suspense thriller ra...
Name: review, Length: 25000, dtype: object

In [31]:
test_reviews

11841    john cassavetes run bottom sees negro sidney p...
19602    movie movie need another western titles sound ...
45519    ethel waters sammy billed without rufus jones ...
25747    find alan jacobs review accurate concerning op...
42642    movie simply although skating montages played ...
                               ...                        
31007    stephen played proctologist one writers passed...
30980    movie stop smiling watching already watched tw...
18418    year rich able travel back time hunt live dino...
29236    really enjoyed usually one fairy tales make be...
37107    next karate excellent martial arts thought eri...
Name: review, Length: 25000, dtype: object

In [32]:
print(y_train.shape, y_test.shape)

(25000, 2) (25000, 2)


In [33]:
y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [21]:
type(train_reviews)

pandas.core.series.Series

In [35]:
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3"
embed = hub.load(url)

In [36]:
X_train = []
for r in tqdm(train_reviews):
    emb = embed(r)
    review_emb = tf.reshape(emb, [-1]).numpy()
    X_train.append(review_emb)

X_train = np.array(X_train)

100%|██████████| 25000/25000 [24:43<00:00, 18.23it/s]


In [37]:
X_test = []
for r in tqdm(test_reviews):
    emb = embed(r)
    review_emb = tf.reshape(emb, [-1]).numpy()
    X_test.append(review_emb)

X_test = np.array(X_test)

100%|██████████| 25000/25000 [25:47<00:00, 16.15it/s]


In [27]:
# module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/4'
# USE_embed = hub.KerasLayer(module_url, trainable=False, name='USE_embedding')

In [38]:
print(X_train.shape, X_test.shape)

(25000, 512) (25000, 512)


In [39]:
print(y_train.shape, y_test.shape)

(25000, 2) (25000, 2)


# Modeling

In [40]:
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout

In [41]:
input_shape = X_train.shape[1:]
X_train.shape[1:]

(512,)

In [42]:
y_train.shape

(25000, 2)

In [55]:
model = tf.keras.Sequential()





model.add(tf.keras.layers.Dense(1000,input_shape=input_shape, activation='relu'))

model.add(tf.keras.layers.Dropout(rate=0.2))

model.add(tf.keras.layers.Dense(units=300,activation='relu'))

model.add(tf.keras.layers.Dense(units=50,activation='relu'))

model.add(tf.keras.layers.Dense(2, activation='sigmoid'))


model.summary()





Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1000)              513000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 300)               300300    
_________________________________________________________________
dense_14 (Dense)             (None, 50)                15050     
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 102       
Total params: 828,452
Trainable params: 828,452
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [59]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), 
                    verbose=1, batch_size=128)

Epoch 1/100
196/196 [==============================] - 1s 5ms/step - loss: 0.1432 - accuracy: 0.9458 - val_loss: 0.6166 - val_accuracy: 0.8061
Epoch 2/100
196/196 [==============================] - 1s 5ms/step - loss: 0.1182 - accuracy: 0.9555 - val_loss: 0.6392 - val_accuracy: 0.8038
Epoch 3/100
196/196 [==============================] - 1s 5ms/step - loss: 0.1055 - accuracy: 0.9600 - val_loss: 0.7478 - val_accuracy: 0.7977
Epoch 4/100
196/196 [==============================] - 1s 5ms/step - loss: 0.0842 - accuracy: 0.9694 - val_loss: 0.7815 - val_accuracy: 0.8042
Epoch 5/100
196/196 [==============================] - 1s 5ms/step - loss: 0.0833 - accuracy: 0.9692 - val_loss: 0.8710 - val_accuracy: 0.7929
Epoch 6/100
196/196 [==============================] - 1s 5ms/step - loss: 0.0719 - accuracy: 0.9729 - val_loss: 0.9227 - val_accuracy: 0.7955
Epoch 7/100
196/196 [==============================] - 1s 5ms/step - loss: 0.0645 - accuracy: 0.9765 - val_loss: 0.8711 - val_accuracy: 0.8032

Epoch 58/100
196/196 [==============================] - 1s 6ms/step - loss: 0.0147 - accuracy: 0.9954 - val_loss: 1.2506 - val_accuracy: 0.8010
Epoch 59/100
196/196 [==============================] - 1s 6ms/step - loss: 0.0159 - accuracy: 0.9942 - val_loss: 1.2270 - val_accuracy: 0.8043
Epoch 60/100
196/196 [==============================] - 1s 6ms/step - loss: 0.0175 - accuracy: 0.9936 - val_loss: 1.1829 - val_accuracy: 0.7986
Epoch 61/100
196/196 [==============================] - 1s 6ms/step - loss: 0.0180 - accuracy: 0.9936 - val_loss: 1.2163 - val_accuracy: 0.8054
Epoch 62/100
196/196 [==============================] - 1s 6ms/step - loss: 0.0116 - accuracy: 0.9961 - val_loss: 1.3559 - val_accuracy: 0.8062
Epoch 63/100
196/196 [==============================] - 1s 5ms/step - loss: 0.0142 - accuracy: 0.9952 - val_loss: 1.2236 - val_accuracy: 0.8023
Epoch 64/100
196/196 [==============================] - 1s 6ms/step - loss: 0.0180 - accuracy: 0.9935 - val_loss: 1.1528 - val_accuracy: